Project 1. Ver 1. Slastnykh Alexander.


Используем домены, дни недели, категории с сайта megaindex. Обучаем нейронную сеть с линейными функциями активации. Есть еще другая версия, где я использую тексты самих  сайтов (10 самых частых слов)

In [119]:
import pandas as pd
import json
from  numpy import array
import numpy as np
import re
from urllib.parse import urlparse
from urllib.parse import unquote
from  collections import Counter
import requests
import datetime

Загрузим предварительно выкаченнные категории доменов. 

In [120]:
df_cat = pd.read_csv('dom_cat.csv',sep = ' ;')

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [121]:
df_cat.head()

,domain,category
0,sotovik.ru,Computers/Mobile/Cellular Computers/Computers/...
1,news.yandex.ru,Employment/Staff Media/Online_Media Media/News...
2,zebra-zoya.ru,Business/Office/General Business/Production/Co...
3,enter.ru,NaN
4,euroavtoprokat.ru,Rest/Travels/Travel_Agencies


Извлечем данные в обычные питоновские списки для удобства работы

In [122]:
cat_l = list(df_cat.category)
dom_l = list(df_cat.domain)

Разберемся с пустыми категориями и порно доменами. Так как они часто заблокированы, то мегаиндекс определяет их как юридические сайты :)

In [196]:
descriptions = []
domains = []

def find_porno(in_str):
    porno_words = ['porn','xvideo','xxx','pussy','pron','nudevista','fuck']
    for w in porno_words:
        if in_str.find(w)>-1:
            return  True
    return False
for i,each in enumerate(cat_l):
    if find_porno(str(dom_l[i])):
        describe = 'Porno'
 
    else:
    
    
        if str(each) == 'nan':
            describe = dom_l[i]
        else:
            describe = str(each).replace('/','').replace('_','') 
        
        
    descriptions.append(describe)
    domains.append(dom_l[i])

In [124]:
df_cat['descriptions']=descriptions

In [125]:
df_cat.head()

,domain,category,descriptions
0,sotovik.ru,Computers/Mobile/Cellular Computers/Computers/...,ComputersMobileCellular ComputersComputersDocu...
1,news.yandex.ru,Employment/Staff Media/Online_Media Media/News...,EmploymentStaff MediaOnlineMedia MediaNewsAgen...
2,zebra-zoya.ru,Business/Office/General Business/Production/Co...,BusinessOfficeGeneral BusinessProductionConsum...
3,enter.ru,NaN,enter.ru
4,euroavtoprokat.ru,Rest/Travels/Travel_Agencies,RestTravelsTravelAgencies


Наши основные исходные данные для работы

In [126]:
df = pd.read_csv('data/gender_age_dataset.txt', sep='\t')

In [127]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


In [128]:
def url2domain(url):
    try:
        url = re.sub('(http(s)*://)+', 'http://', url)
        parsed_url = urlparse(unquote(url.strip()))
        if parsed_url.scheme not in ['http','https']: 
            return None
        netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
        if netloc is not None: 
            return str(netloc).strip()
        return None
    except:
        return None
    

In [197]:
def ts2dayhour(timestamp): #преобразуем таймстемп в день недели. Час я решил не использовать, так как он ухудшает результат.
    tms = int(timestamp)/1000
    return str(datetime.datetime.fromtimestamp(tms).strftime('%A')) # %A %Hh 

In [198]:
ts2dayhour(1426661722000) #пример работы

'Wednesday'

In [199]:
def filltimes(visits): #проходит по всем таймстемпам пользователя
    timestring=''
    for each in visits['visits']:        
        timestring+=ts2dayhour(each['timestamp'])+' '
    return timestring

In [200]:
def to_counter(visits): #на вход json с посещениями, на выход счетчик доменов. Осталось от первоначальной задумки. 
    # естественно можно упростить, в конечную версию пошли просто домены, без счетчиков
    c = Counter()
    for each in visits['visits']:
        
        c[url2domain(str(each['url']))]+=1
        
    return c

In [201]:
gender_l = list(df['gender']) #выгружаем данные в питоновские списки, так удобнее чем df работать
age_l = list(df['age'])
user_json_l = list(df['user_json'])

In [202]:
#посмотрим как выглядят сырые данные о посещениях
user_json_l[0]

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

Далее проводится основная работа по созданию списка, где будут храниться данные для обучения. Это будут категории доменов, день недели посещения, сами домены.

In [163]:
visits_l = [] 
domain_map = []
j=0
print(len(user_json_l))
for each in user_json_l:
    try:
        describe = ''
        visit = json.loads(each)
        visits = to_counter(visit).most_common(50)
        describe +=filltimes(visit)
        for u in visits:
            url_str=u[0].strip()
            if url_str not in domains:
                describe += url_str+' '
            else:
                describe +=descriptions[domains.index(url_str)] + ' ' + url_str+' '
        j+=1
        
        describe=' '.join(list(set(describe.split()))) #избавляемся от дублей. через превращение в set и обратно
        
        visits_l.append(describe)
        
        if j%1000 == 0:
            print(str(j),end=' ')
    except:
        visits_l.append('')

41138
1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 

In [204]:
visits_l[0] #пример одной записи о пользователи, которая пойдет на обучение

'ComputersMobileCellular zebra-zoya.ru news.yandex.ru Saturday EmploymentStaff MediaNewsAgencies ComputersMobileGeneral sotovik.ru ComputersComputersDocumentation MediaOnlineMedia Wednesday BusinessOfficeGeneral BusinessProductionConsumables BusinessOfficeStationary'

Эту работу я сделал первым, поэтому использовал другую библиотеку. Это keras для нейронных сетей, в ней же есть весь инструментарий для текстовой обработки. По смыслу все тоже самое, что мы учили.

In [165]:
from keras.preprocessing.text import Tokenizer 

In [205]:
tokenizer = Tokenizer(num_words=30000) #инициируем токенайзер, будем использовать 30 тыс самых частых слов
tokenizer.fit_on_texts(visits_l) #даем ему команду составить словарь


In [167]:
textSequences = tokenizer.texts_to_sequences(visits_l) #превращаем слова в токены по словарю

In [168]:
textSequences[0] #пример

[109,
 8484,
 18203,
 1,
 24,
 14,
 1,
 9,
 54,
 26,
 208,
 4692,
 1,
 180,
 16,
 8,
 89,
 67,
 484]

Разворачиваем наши фичи в бинарные вектора.

In [169]:
features = tokenizer.sequences_to_matrix(textSequences, mode='binary') # варианты binary tfidf count freq

In [170]:
features[0] #пример

array([ 0.,  1.,  0., ...,  0.,  0.,  0.])

Далее создадим список с целями (labels) для обучения. Попутно выделим в отдельный список super_test_l c фичами для финальной выгрузки (то есть те, у которых не задан пол возвраст)

In [206]:
label_l = [] #цели обучения
super_test_l = [] #список фич для финального результата - что будем выгружать в лк

s_index=0
age_voc = {'18-24':0,'25-34':1,'35-44':2,'45-54':3,'>=55':4}
for i,a in enumerate(age_l):
    label_vec = np.zeros(10)
    if a!='-':
        age_shift = age_voc[a]
    g = gender_l[i]
    if g == 'M': #здесь мы создаем выход из 10 категорий
        label_vec[age_shift]=1
        label_l.append(label_vec) #1
        
    elif g == 'F':
        label_vec[age_shift+5] =1
        label_l.append(label_vec) #-1
        
    if a=='-' and s_index ==0:
        s_index=i
    if i%2000 == 0:
        print(i,end=' ')
        print(label_l[-1])
if s_index>0:
    super_test_l = features[s_index:]
    features = features[:s_index]

0 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
2000 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
4000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
6000 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
8000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
10000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
12000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
14000 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
16000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
18000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
20000 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
22000 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
24000 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
26000 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
28000 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
30000 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
32000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
34000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
36000 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
38000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
40000 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]


In [172]:
features[0:100] #пример  матрицы для обучения. это не частоты, это бинарные признаки!

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.]])

In [173]:
#просто копии
train_features = features 
train_labels=array(label_l)

In [174]:
len(tokenizer.word_index) #а это общая длина нашего словаря

77072

In [175]:
len(train_features[0]) #еще раз проверяем размер одного учебного вектора. 

30000

In [176]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import mean_squared_error

Далее пойдет само обучение. Оно выполняется на сетке. 30000 входов. 3 х 10 слоя сделал специально, чтобы показать как это делается. Для этой задачи достаточно сети из  3 скрытых нейронов!

In [184]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf
from keras.backend import tensorflow_backend as K

with tf.Session(config=tf.ConfigProto(
                    intra_op_parallelism_threads=48)) as sess:
    K.set_session(sess)
   
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import Activation
    from keras.layers import Dropout,Conv1D,MaxPooling1D,Embedding,Flatten,LSTM
    

    neurons = 10
    epochs = 5
    

    print('Собираем модель...',neurons)
    total_words = len(tokenizer.word_index)
    model = Sequential()
    #Добавляем один  слой из neurons нейронов и вход  = длине вектора
    model.add(Dense(neurons, input_shape=(len(features[0]),)))
    model.add(Activation('linear')) #нормальная функция активации relu ! в этом случае опытным путем выбрал линейную
    model.add(Dropout(0.1))

    #еще скрытый слой
    model.add(Dense(10)) 
    model.add(Activation('linear'))
    #model.add(Dropout(0.1))

    #и еще
    model.add(Dense(10))
    model.add(Activation('linear'))
    #model.add(Dropout(0.1))

    #а это выход - 10 нейронов по  количеству категорий. софтмакс означает что один из них будет "ярче" чем другие
    #это и будет искомая категория
    model.add(Dense(10))
    model.add(Activation('softmax'))
    #Варианты оптимизатора: adamax adadelta rmsprop ... - разница есть, выбирал опытным путем!
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta', 
                  metrics=['accuracy'])




    history = model.fit(train_features, train_labels,
                    batch_size=64,
                    epochs=epochs,
                    verbose=2,
                    validation_split=0.1)
    #validation_split=0.1 указывает на использование 10% тестовой выборки. отдельно ее создавать не надо. очень удобно

    train_predicted_labels = model.predict(train_features) 


    train_error= mean_squared_error(train_labels,train_predicted_labels)) #средняя ошибка обучения, можно использовать что  то другое
    out_predicted_labels = model.predict(array(super_test_l)) #это наш результат
    print('MSE',train_error)
    print('Done!!!')
    

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9773449083379125478
]
Собираем модель... 10
Train on 32524 samples, validate on 3614 samples
Epoch 1/5
15s - loss: 2.0412 - acc: 0.2724 - val_loss: 1.9540 - val_acc: 0.3077
Epoch 2/5
10s - loss: 1.8971 - acc: 0.3178 - val_loss: 1.9206 - val_acc: 0.3094
Epoch 3/5
10s - loss: 1.8454 - acc: 0.3379 - val_loss: 1.9083 - val_acc: 0.3099
Epoch 4/5
10s - loss: 1.8069 - acc: 0.3528 - val_loss: 1.9074 - val_acc: 0.3077
Epoch 5/5
10s - loss: 1.7721 - acc: 0.3655 - val_loss: 1.9063 - val_acc: 0.3149

Done!!!


Результат 0.31+ на всей выборке. Это почти тоже самое, что даст чекер из ЛК. Неплохо уменьшить выборку вдвое, тем самым повысив ее качество. А также превратить данные в нужный для чекера вид. Как показала практика - урезание выборки вдвое позволяет поднять результат с 0.31 по всей до 0.36+ для половины.

Наши бинарные вектора ответов надо превратить обратно в человеческие буквы и цифры и выгрузить в файл как финальный результат работы. Попутно избавляемся от тех, где сеть не очень уверена и увеличить общий рейтинг решения. В ответ мы получаем значение 10-ти нейронов, у одного из них наибольшее значение. Это и есть наша категория. Я смотрю насколько его значение больше по отношению к другим, то есть  насколько его значение (оно максимальное) больше следующего по степени. Эту разницу я решил назвать альфой. Если она недостаточно большая - значит сеть неуверена. Насколько большая - очень просто, мы хотим максимизировать результат, поэтому просто подбираем альфу так, чтобы в итоговую выборку попало 50% пользователей. 

In [185]:
def num_to_cat(in_list,alfa): # на вход получает вектор из десятка цифр. там где большая там категория. если разнца достато
    pos = list(in_list).index(max(in_list)) 
    if max(in_list)-sorted(in_list)[-2]>alfa: # разница определяется альфой
        cat_voc = {0:'M 18-24',
                    1:'M 25-34',
                    2:'M 35-44',
                    3:'M 45-54',
                    4:'M >=55',
                    5:'F 18-24',
                    6:'F 25-34',
                    7:'F 35-44',
                    8:'F 45-54',
                    9:'F >=55'}
        return cat_voc[pos]
    else:
        return '- -'

In [189]:

alfa = 0.0745 #альфу подбираю руками. это надо делать автоматом, но было влом писать.

out_g_l = [] #out gengeder list
out_a_l = [] #out agelist
for i,each in enumerate(out_predicted_labels): #перебираем  все предсказанные лейблы
    if sum([1 for x in super_test_l[i] if x>0])>4: #здесь мы пытаемся убрать тех у кого слишком мало данных
        g,a = num_to_cat(each,alfa).split()
    else:
        g=a='-'
    out_g_l.append(g)
    out_a_l.append(a)
print(sum([1 for x in out_g_l if x!='-'])/len(out_g_l),'доля пользователей с определенным полом и возрастом')

0.5012 доля пользователей с определенным полом и возрастом


In [190]:
len(out_g_l) #проверим что ничего не перепутали с результатом.

5000

Соберем df на выгрузку в файл ответа

In [192]:
df2=df[df.gender=='-']
del df2['age']
del df2['gender']
del df2['user_json']
df2['gender']= out_g_l
df2['age'] = out_a_l

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [193]:
df2=df2.sort_values('uid')
len(df2)

5000

In [194]:
df2.head() #результат

,uid,gender,age
37813,0000e7ca-32e6-4bef-bdca-e21c025071ff,F,25-34
37814,0000f3cf-6e9a-4eab-92f4-cefdad108c83,M,25-34
37815,000381a6-0400-40f8-98c0-93a6c0852d2d,-,-
37816,00062d89-a3ba-40dd-9a25-212c9f429133,-,-
37817,0009233e-a0c2-438e-ac36-bd3e1b7dad1e,M,25-34


In [195]:
df2.to_csv('project01_gender-age.csv',index=False,sep='\t')


time: 2017-08-11 14:08
correctly predicted users / total number of users: 0.184
part of users with predicted gender + age: 0.5012
correctly predicted users / number of predicted users: 0.367118914605
resulting csv is readable and contains all necessary uids: True
file project01_gender-age.csv exists in your dir: True